In [1]:
from sqlalchemy import create_engine
import pymysql

sqlEngine       = create_engine('mysql+pymysql://root:@127.0.0.1/kaggle_hour_price', pool_recycle=3600)
dbConnection    = sqlEngine.connect()

In [2]:
%run package_import.ipynb

In [3]:
import os
os.chdir(path='/Users/liuyifu/Project_data')

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, SVR
from sklearn import preprocessing
import sklearn.metrics as sm
from sklearn import linear_model

In [5]:
df_combined = pd.read_sql_query(

''' 
select *
from transformed_combined
'''
,dbConnection)

In [134]:
df_profile = pd.read_sql_query(

''' 
select *
from raw_data_profile
'''
,dbConnection)

df_profile.sort_values(by='column_names')

,column_names,col_desc,value_desc,data_type,unique_count,min_value,max_value,mean_value,median_value,null_values,total_rows,%_null_values,num_lower_outliers,num_higher_outliers
43,1stFlrSF,First Floor square feet,{},int64,1083,334.0,5095.0,1159.581706,1082.0,0,2919,0.000000,0.0,42.0
44,2ndFlrSF,Second floor square feet,{},int64,635,0.0,2065.0,336.483727,0.0,0,2919,0.000000,0.0,8.0
69,3SsnPorch,Three season porch area in square feet,{},int64,31,0.0,508.0,2.602261,0.0,0,2919,0.000000,0.0,37.0
6,Alley,Type of alley access to property,"{'Grvl': 'Gravel', 'Pave': 'Paved', 'NA': 'No alley access'}",object,3,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
51,BedroomAbvGr,None,nan,int64,8,0.0,8.0,2.860226,3.0,0,2919,0.000000,8.0,70.0
15,BldgType,Type of dwelling,"{'1Fam': 'Single-family Detached', '2FmCon': 'Two-family Conversion; originally built as one-family dwelling', 'Duplx': 'Duplex', 'TwnhsE': 'Townhouse End Unit', 'TwnhsI': 'Townhouse Inside Unit'}",object,5,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
31,BsmtCond,Evaluates the general condition of the basement,"{'Ex': 'Excellent', 'Gd': 'Good', 'TA': 'Typical - slight dampness allowed', 'Fa': 'Fair - dampness or some cracking or settling', 'Po': 'Poor - Severe cracking, settling, or wetness', 'NA': 'No Basement'}",object,5,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
32,BsmtExposure,Refers to walkout or garden level walls,"{'Gd': 'Good Exposure', 'Av': 'Average Exposure (split levels or foyers typically score average or above)', 'Mn': 'Mimimum Exposure', 'No': 'No Exposure', 'NA': 'No Basement'}",object,5,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
34,BsmtFinSF1,Type 1 finished square feet,{},float64,992,0.0,5644.0,441.423235,368.5,1,2919,0.000343,0.0,15.0
36,BsmtFinSF2,Type 2 finished square feet,{},float64,273,0.0,1526.0,49.582248,0.0,1,2919,0.000343,0.0,347.0


### Read train data

In [6]:
df_combined = pd.read_sql_query(

''' 
select *
from transformed_combined
'''
,dbConnection)

# df_combined.drop(columns=['source'], inplace=True)


### Prepare data for ML

In [18]:
def preprocessing_data(input: pd.DataFrame) -> np.array:
    """[summary]

    Args:
        input (pd.DataFrame): [description]

    Returns:
        np.array: [description]
    """
    
    train_rows = input[input['source']=='train'].shape[0]
    
    cols_selected = [
            'LotArea', 'LandContour', 'Utilities',
            'LandSlope', 'BldgType', 'OverallQual',
            'OverallCond', 'ExterQual', 'ExterCond', 
            'MoSold','YearBuilt', 'SaleType', 
            'PoolArea', 'GarageArea', 'Heating', 'CentralAir', 
            'Alley', 'Street', 'TotalBsmtSF', 'Neighborhood',
            '1stFlrSF', '2ndFlrSF',
             'SalePrice'        
                    ]
    input = input[cols_selected]
    X = input.iloc[:, :-1]
    y = input.iloc[:, -1]

    X = pd.get_dummies(X, dummy_na=False)
    X.drop(columns=['LandSlope_Gtl', 'ExterQual_TA', 'ExterCond_Gd',
                    'SaleType_WD', 'Utilities_AllPub' ,'CentralAir_N', 'Heating_GasA', 
                    'Alley_No alley access', 'Utilities_NoSeWa', 'Heating_Floor',
                    'Heating_OthW',], inplace=True)

    X_train = X.iloc[:train_rows,:]
    X_test = X.iloc[train_rows:,:]

    y_train = y.iloc[:train_rows,]
    y_test = y.iloc[train_rows:,]
    
    return X_train, y_train, X_test, y_test


In [19]:
X_train, y_train, X_test, y_test = preprocessing_data(df_combined)

In [20]:
def color_highlight_highly_corr(val):

    threshold = 0.7

    if abs(val) >= threshold:
        color = 'red'
    else:
        color = 'black'

    return 'color: %s' % color

X_test.corr().style.applymap(color_highlight_highly_corr)

,LotArea,OverallQual,OverallCond,MoSold,YearBuilt,PoolArea,GarageArea,TotalBsmtSF,1stFlrSF,2ndFlrSF,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LandSlope_Mod,LandSlope_Sev,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterCond_Ex,ExterCond_Fa,ExterCond_Po,ExterCond_TA,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,Heating_GasW,Heating_Grav,Heating_Wall,CentralAir_Y,Alley_Grvl,Alley_Pave,Street_Grvl,Street_Pave,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
LotArea,1.000000,0.107989,-0.103402,0.005152,0.048314,0.140494,0.315841,0.283049,0.456417,-0.007862,0.040427,0.021220,0.199463,-0.124336,0.044426,0.130618,0.365953,0.009992,-0.030879,-0.298982,-0.292862,0.133377,-0.014880,-0.005520,0.013229,0.054169,0.009894,-0.034114,-0.006927,0.008684,0.025970,-0.007822,-0.026962,-0.020857,0.059571,-0.001203,0.062161,0.001109,0.013050,0.062355,-0.064368,-0.142231,0.029291,-0.029291,-0.112920,-0.109304,-0.157630,-0.126406,0.094898,0.039848,0.064980,0.036128,0.075247,-0.020921,-0.187836,0.108935,0.009908,-0.146888,0.066790,0.042621,0.086522,-0.096067,-0.070524,0.022332,0.023859,-0.129363,0.061134,0.073113,0.106387
OverallQual,0.107989,1.000000,-0.095874,-0.008764,0.621910,-0.012330,0.568161,0.560314,0.481842,0.195869,-0.051201,0.204262,-0.033358,-0.098525,0.027084,0.008043,-0.023885,-0.110666,-0.114505,-0.023407,0.192968,0.417569,-0.182916,0.535539,0.020055,-0.180688,-0.079418,0.198550,-0.115707,0.008849,-0.002491,-0.112680,-0.002877,-0.076231,0.349259,-0.048508,-0.034796,-0.027828,-0.105213,0.300713,-0.150753,0.082206,-0.085549,0.085549,0.061416,0.034698,-0.020087,-0.115260,0.003385,0.133144,0.025485,-0.193217,0.069337,-0.189801,-0.146004,-0.085863,-0.225122,0.023968,0.040125,0.187078,0.392760,-0.222430,-0.056748,-0.160218,0.040867,0.254896,0.234273,0.124370,0.101435
OverallCond,-0.103402,-0.095874,1.000000,-0.009767,-0.361027,-0.037191,-0.156769,-0.177190,-0.170451,-0.019316,0.072047,-0.065578,0.008270,-0.002115,0.008593,0.004601,0.132586,-0.004986,-0.087587,-0.011028,-0.114185,-0.072593,-0.101460,-0.170563,0.149511,-0.174036,-0.068345,-0.257172,-0.076915,0.021421,0.004601,-0.059745,-0.002534,-0.008989,-0.140072,0.009240,0.015849,-0.035068,-0.035068,0.121102,0.110132,-0.048911,0.006514,-0.006514,-0.043354,0.013085,-0.011072,0.126255,-0.022822,-0.083404,0.080366,0.002363,-0.111421,0.038411,0.015481,-0.008943,0.105805,0.007873,0.043732,-0.050382,-0.124209,0.193346,-0.023414,0.025764,-0.060877,-0.129528,-0.067002,-0.036029,0.011795
MoSold,0.005152,-0.008764,-0.009767,1.000000,0.015599,-0.055731,0.052470,0.021525,0.048064,-0.009415,0.017841,-0.033340,-0.022770,0.022038,-0.053585,0.020501,-0.029581,-0.033583,0.058572,0.023527,0.001157,0.033414,-0.002512,-0.014472,-0.038398,-0.023520,-0.055873,0.044498,-0.011168,0.021031,-0.018417,-0.004156,0.022075,0.003822,0.084209,-0.030906,0.029149,-0.035453,0.012195,-0.025278,0.005547,-0.038213,-0.030007,0.030007,0.022867,0.024441,0.019483,-0.059817,-0.030632,-0.002957,0.007584,-0.011071,-0.007442,0.004150,-0.026172,0.003941,0.041178,0.014316,0.014124,0.012200,-0.009757,0.005266,-0.057413,-0.007910,-0.001061,0.025394,0.038627,-0.017598,-0.059910
YearBuilt,0.048314,0.621910,-0.361027,0.015599,1.000000,-0.001060,0.482497,0.425447,0.338733,0.025195,-0.175111,0.082019,-0.016600,0.058434,0.004719,-0.013982,-0.126620,-0.176801,-0.025779,0.071269,0.243803,0.220482,-0.157127,0.543415,-0.018217,-0.208547,

In [21]:
X_train, X_validate, y_train, y_validate = train_test_split(
    X_train, y_train, test_size=0.2, random_state=5   
)

#### Linear Regression

In [12]:
linear_regressor = linear_model.LinearRegression()
linear_regressor.fit(X_train, y_train)
y_test_pred = linear_regressor.predict(X_validate)

In [10]:
def accuracy(data: list, predict_data: list):

    print(f'Mean absolute error: {sm.mean_absolute_error(predict_data, data)}')
    print(f'Mean square error: {sm.mean_squared_error(predict_data, data)}')
    print(f'Median absolute error: {sm.median_absolute_error(predict_data, data)}')
    print(f'Explained variance score: {sm.explained_variance_score(predict_data, data)}')
    print(f'R2 score: {sm.r2_score(predict_data, data)}')

In [14]:
accuracy(y_test_pred, y_validate)

Mean absolute error: 19818.250503417294
Mean square error: 1054495557.3477666
Median absolute error: 13322.366987005167
Explained variance score: 0.8376989812060145
R2 score: 0.8375201101359835


#### Ridge regression

In [22]:
ridge_regressor = linear_model.Ridge(alpha=10, fit_intercept=True, max_iter=10000)
ridge_regressor.fit(X_train, y_train)
y_test_pred = ridge_regressor.predict(X_validate)

In [23]:
accuracy(y_test_pred, y_validate)

Mean absolute error: 18533.821181091767
Mean square error: 962516183.0148598
Median absolute error: 12879.741386710317
Explained variance score: 0.8517594879926353
R2 score: 0.8516925725112262


#### Polynomial regression

In [49]:
polynomial = preprocessing.PolynomialFeatures(degree=4)
X_train_transformed = polynomial.fit_transform(X_train)

poly_linear_model = linear_model.LinearRegression()
poly_linear_model.fit(X_train_transformed, y_train)

X_validate_transformed = polynomial.fit_transform(X_validate)
y_test_pred = poly_linear_model.predict(X_validate_transformed)

In [18]:
accuracy(y_test_pred, y_validate)

Mean absolute error: 4211813.223930025
Mean square error: 1282351966971781.5
Median absolute error: 458357.24549307395
Explained variance score: -196404.73225748667
R2 score: -197587.7000268938


#### Predict test data

In [35]:
X_test[X_test.isna().values.any(axis=1)]

,LotArea,OverallQual,OverallCond,MoSold,YearBuilt,PoolArea,GarageArea,TotalBsmtSF,1stFlrSF,2ndFlrSF,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LandSlope_Mod,LandSlope_Sev,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterCond_Ex,ExterCond_Fa,ExterCond_Po,ExterCond_TA,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,Heating_GasW,Heating_Grav,Heating_Wall,CentralAir_Y,Alley_Grvl,Alley_Pave,Street_Grvl,Street_Pave,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
2120,5940,4,7,4,1946,0,280.0,NaN,896,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2576,9060,5,6,3,1923,0,NaN,859.0,942,886,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
X_test['GarageArea'].fillna(0, inplace=True)
X_test['TotalBsmtSF'].fillna(0, inplace=True)
y_test_pred = ridge_regressor.predict(X_test)

In [38]:
y_test_pred

array([132037.92750039, 162596.71265137, 169826.51911339, ...,
       155752.74179236, 121411.30711214, 230306.4957054 ])

In [51]:
df_submission = pd.read_csv('House_Prices/sample_submission.csv')

In [52]:
df_y_test_pred = pd.DataFrame(y_test_pred)

In [53]:
df_submission['SalePrice'] = df_y_test_pred

In [54]:
df_submission.to_csv('House_Prices/submission/house_price_r2_0.85.csv',index=False)